In [1]:
from hypex.dataset import Dataset, ExperimentData, InfoRole, FeatureRole, TreatmentRole, TargetRole
from hypex.experiments.aa import AA_TEST
from hypex.reporters import AADictReporter

In [2]:
data = Dataset(
    roles={
        "user_id": InfoRole(),
        "signup_month": FeatureRole(),
        "treat": TreatmentRole(),
        "pre_spends": TargetRole(),
        "post_spends": TargetRole(),
        "age": FeatureRole(),
        "gender": FeatureRole(),
        "industry": FeatureRole(),
    }, data="data.csv",
)
data

      user_id  signup_month  treat  pre_spends  post_spends   age gender  \
0           0             0      0       488.0   414.444444   NaN      M   
1           1             8      1       512.5   462.222222  26.0    NaN   
2           2             7      1       483.0   479.444444  25.0      M   
3           3             0      0       501.5   424.333333  39.0      M   
4           4             1      1       543.0   514.555556  18.0      F   
...       ...           ...    ...         ...          ...   ...    ...   
9995     9995            10      1       538.5   450.444444  42.0      M   
9996     9996             0      0       500.5   430.888889  26.0      F   
9997     9997             3      1       473.0   534.111111  22.0      F   
9998     9998             2      1       495.0   523.222222  67.0      F   
9999     9999             7      1       508.0   475.888889  38.0      F   

        industry  
0     E-commerce  
1     E-commerce  
2      Logistics  
3     E-com

In [3]:
data.roles

{'user_id': Info,
 'signup_month': Feature,
 'treat': Treatment,
 'pre_spends': Target,
 'post_spends': Target,
 'age': Feature,
 'gender': Feature,
 'industry': Feature}

In [4]:
test = AA_TEST
ed = ExperimentData(data)
result = test.execute(ed)

In [5]:
AADictReporter().report(result)

{'random_state': None,
 'B pre_spends control mean': 486.9008,
 'B pre_spends difference': 0.38589999999999236,
 'B pre_spends difference %': 0.07925639062411616,
 'B pre_spends test mean': 487.2867,
 'B post_spends control mean': 452.1402222222222,
 'B post_spends difference': 0.04866666666674746,
 'B post_spends difference %': 0.010763622494702751,
 'B post_spends test mean': 452.1888888888889,
 'B control size': 5000.0,
 'B control size %': 50.0,
 'B test size': 5000.0,
 'B test size %': 50.0,
 'TTest p-value': 0.6286670862348981,
 'TTest pass': 0.0,
 'KSTest p-value': 0.7095449603070298,
 'KSTest pass': 0.0,
 'mean test score': 0.6825856689496526}

In [6]:
result.analysis_tables

{'GroupSizes╰╰[][A]':                      B
 control size    5000.0
 control size %    50.0
 test size       5000.0
 test size %       50.0,
 "GroupDifference╰╰['pre_spends'][A]":                                   B
 pre_spends control mean  486.900800
 pre_spends difference      0.385900
 pre_spends difference %    0.079256
 pre_spends test mean     487.286700,
 "TTest╰╰['pre_spends'][A]":   group  statistic   p-value   pass
 0     B  -1.022496  0.306571  False,
 "KSTest╰╰['pre_spends'][A]":   group  statistic   p-value   pass
 0     B     0.0166  0.496232  False,
 "GroupDifference╰╰['post_spends'][A]":                                    B
 post_spends control mean  452.140222
 post_spends difference      0.048667
 post_spends difference %    0.010764
 post_spends test mean     452.188889,
 "TTest╰╰['post_spends'][A]":   group  statistic   p-value   pass
 0     B   -0.06175  0.950763  False,
 "KSTest╰╰['post_spends'][A]":   group  statistic   p-value   pass
 0     B      0.011  0.922

In [7]:
from hypex.experiments.ab import AB_TEST

test = AB_TEST
ed = ExperimentData(data)
result = test.execute(ed)

In [8]:
result.analysis_tables['ABAnalyzer╰╰']

   TTest p-value  TTest pass  UTest ['pre_spends'][t]  \
0   1.157524e-30         1.0                        1   

   UTest ['post_spends'][t]  ATE ['pre_spends'][t]  ATE ['post_spends'][t]  
0                         1              487.09375              452.164556  

In [9]:
from hypex.experiments.homogeneity import HOMOGENEITY_TEST

test = HOMOGENEITY_TEST
ed = ExperimentData(data)
result = test.execute(ed)

In [10]:
result.analysis_tables

{"GroupDifference╰╰['pre_spends'][t]":                                   1
 pre_spends control mean  484.911973
 pre_spends difference      4.308406
 pre_spends difference %    0.888492
 pre_spends test mean     489.220379,
 "TTest╰╰['pre_spends'][t]":    group  statistic       p-value  pass
 0      1 -11.489293  2.315047e-30  True,
 "KSTest╰╰['pre_spends'][t]":    group  statistic       p-value  pass
 0      1   0.077573  1.559150e-13  True,
 "GroupDifference╰╰['post_spends'][t]":                                    1
 post_spends control mean  420.046619
 post_spends difference     63.424045
 post_spends difference %   15.099287
 post_spends test mean     483.470664,
 "TTest╰╰['post_spends'][t]":    group   statistic  p-value  pass
 0      1 -135.560001      0.0  True,
 "KSTest╰╰['post_spends'][t]":    group  statistic  p-value  pass
 0      1     0.8959      0.0  True,
 'OneAASplitAnalyzer╰╰':    TTest p-value  TTest pass  KSTest p-value  KSTest pass  mean test score
 0   1.157524e-3

In [11]:
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

from hypex.comparators import TTest, KSTest, GroupSizes, GroupDifference
from hypex.utils import SpaceEnum
from hypex.splitters import AASplitter
from hypex.analyzers import OneAASplitAnalyzer
from hypex.experiments import CycledExperiment, Experiment, OnRoleExperiment

aa = Experiment(
    executors=[
        AASplitter(),
        OnRoleExperiment(
            executors=[
                GroupSizes(grouping_role=TreatmentRole(), space=SpaceEnum.additional),
                GroupDifference(grouping_role=TreatmentRole(), space=SpaceEnum.additional),
                TTest(grouping_role=TreatmentRole(), space=SpaceEnum.additional),
                KSTest(grouping_role=TreatmentRole(), space=SpaceEnum.additional),
            ],
            role=TargetRole(),
        )
    ]
)
experiment = CycledExperiment(n_iterations=20, inner_executor=aa, analyzer=OneAASplitAnalyzer())
ed = ExperimentData(data)
res = experiment.execute(ed)

In [12]:
res.additional_fields

     AASplitter╰╰0 AASplitter╰╰1 AASplitter╰╰2 AASplitter╰╰3 AASplitter╰╰4  \
0                B             A             A             A             A   
1                A             A             A             A             B   
2                B             B             B             A             B   
3                B             B             B             B             B   
4                B             A             B             B             A   
...            ...           ...           ...           ...           ...   
9995             B             A             A             B             B   
9996             A             A             A             A             A   
9997             B             B             B             A             B   
9998             A             B             B             B             B   
9999             B             B             B             A             A   

     AASplitter╰╰5 AASplitter╰╰6 AASplitter╰╰7 AASplitter╰╰8 AA

In [13]:
res.analysis_tables

{"GroupSizes╰╰['pre_spends'][A]0":                      B
 control size    5000.0
 control size %    50.0
 test size       5000.0
 test size %       50.0,
 "GroupDifference╰╰['pre_spends'][A]0":                                   B
 pre_spends control mean  487.430000
 pre_spends difference     -0.672500
 pre_spends difference %   -0.137969
 pre_spends test mean     486.757500,
 "TTest╰╰['pre_spends'][A]0":   group  statistic   p-value   pass
 0     B   1.782072  0.074768  False,
 "KSTest╰╰['pre_spends'][A]0":   group  statistic   p-value   pass
 0     B     0.0186  0.352691  False,
 "GroupSizes╰╰['post_spends'][A]0":                      B
 control size    5000.0
 control size %    50.0
 test size       5000.0
 test size %       50.0,
 "GroupDifference╰╰['post_spends'][A]0":                                    B
 post_spends control mean  453.066267
 post_spends difference     -1.803422
 post_spends difference %   -0.398048
 post_spends test mean     451.262844,
 "TTest╰╰['post_spends']

In [14]:
res_analyzer = OneAASplitAnalyzer().execute(res)

In [15]:
res_analyzer.analysis_tables['OneAASplitAnalyzer╰╰']

   TTest p-value  TTest pass  KSTest p-value  KSTest pass  mean test score
0       0.503151       0.025         0.60017          0.0          0.56783

In [16]:
# analyser version

from hypex.dataset import StatisticRole

new_res = {}
fields = ['pre_spends', 'post_spends'] # получаем столбцы по TargetRole
tests = ['KSTest', 'TTest']
for j in fields:
    for i in list(res.analysis_tables.keys()):
        if j in i and i.split('╰╰')[0] in tests:
            test = i.split('╰╰')[0]
            if j not in new_res:
                new_res[j] = {test: []}
            elif test not in new_res[j]:
                new_res[j].update({test: []})
            new_res[j][test].append(list(res.analysis_tables[i].data['pass'])[0])
    new_res[j] = Dataset.from_dict(new_res[j], roles={}).mean()
    new_res[j].add_column(new_res[j].apply(
                lambda x: int(0.8 * 0.05 <= x['TTest'] <= 1.2 * 0.05), {'TTest passed': StatisticRole()}, axis=1
            ))
    new_res[j].add_column(new_res[j].apply(
                lambda x: int(0.8 * 0.05 <= x['KSTest'] <= 1.2 * 0.05), {'KSTest passed': StatisticRole()}, axis=1
            ))
new_res

{'pre_spends':       KSTest  TTest  TTest passed  KSTest passed
 mean     0.0    0.0             0              0,
 'post_spends':       KSTest  TTest  TTest passed  KSTest passed
 mean     0.0   0.05             1              0}

In [17]:
import pandas as pd

t = pd.read_csv("data.csv")
t.sample(frac=1, random_state=None)

,user_id,signup_month,treat,pre_spends,post_spends,age,gender,industry
7463,7463,0,0,493.0,414.888889,26.0,M,E-commerce
5717,5717,0,0,470.0,423.666667,43.0,M,E-commerce
6900,6900,0,0,462.5,416.333333,NaN,M,E-commerce
2927,2927,0,0,509.5,425.222222,54.0,F,Logistics
1902,1902,0,0,519.0,418.555556,52.0,M,Logistics
...,...,...,...,...,...,...,...,...
4108,4108,5,1,461.0,507.111111,26.0,M,E-commerce
6729,6729,1,1,527.5,518.777778,35.0,M,Logistics
4152,4152,0,0,496.0,424.000000,67.0,F,Logistics
9220,9220,6,1,488.0,474.555556,NaN,M,Logistics


In [1]:
from hypex.hypotheses.hypothesis import Hypothesis
ed, exp  = Hypothesis("test_config.json").execute()

In [ ]:
res = exp.execute(ed)

In [3]:
exp.executors[1].executors

In [7]:
res.additional_fields

     AASplitter╰╰
0               A
1               A
2               A
3               B
4               B
...           ...
9995            B
9996            A
9997            A
9998            B
9999            A

[10000 rows x 1 columns]